In [3]:
import csv
import os
import time

In [2]:
def row_to_edge(row):
    """
    Given an election result row or poll data row, returns the Democratic edge
    in that state.
    """
    return float(row["Dem"]) - float(row["Rep"])  

In [3]:
def state_edges(election_result_rows):
    """
    Given a list of election result rows, returns state edges.
    The input list does has no duplicate states;
    that is, each state is represented at most once in the input list.
    """
    dic={}
    for i in election_result_rows:
        dic.update({ i["State"]: row_to_edge(i)})
    return dic

In [4]:
rows1 = [{'State': 'WA', 'Dem': '1.0', 'Rep': '0.1'}]

In [5]:
state_edges(rows1)

{'WA': 0.9}

In [6]:
rows2 = [{'State': 'WA', 'Dem': '1.0', 'Rep': '0.1'},
               {'State': 'CA', 'Dem': '0.2', 'Rep': '1.3'}]

In [7]:
state_edges(rows2)

{'WA': 0.9, 'CA': -1.1}

In [8]:
next(item for item in rows2 if item["State"] == "WA")

{'State': 'WA', 'Dem': '1.0', 'Rep': '0.1'}

In [9]:
def earlier_date(date1, date2):
    """
    Given two dates as strings (formatted like "Oct 06 2012"), returns True if 
    date1 is after date2.
    """
    return (time.strptime(date1, "%b %d %Y") < time.strptime(date2, "%b %d %Y"))

def most_recent_poll_row(poll_rows, pollster, state):
    """
    Given a list of poll data rows, returns the most recent row with the
    specified pollster and state. If no such row exists, returns None.
    """
    recent_date = "Oct 01 1500"
    data = None
    for i in poll_rows:
        if i["State"] == state and i["Pollster"] == pollster:
            if earlier_date(recent_date, i["Date"]):
                recent_date = i["Date"]
                data = i
    return data

In [10]:
poll_rows1 = [{"ID":1, "State":"WA", "Pollster":"A", "Date":"Jan 07 2010"},
              {"ID":2, "State":"WA", "Pollster":"B", "Date":"Mar 21 2010"},
              {"ID":3, "State":"WA", "Pollster":"A", "Date":"Jan 08 2010"},
              {"ID":4, "State":"OR", "Pollster":"A", "Date":"Feb 10 2010"},
              {"ID":5, "State":"WA", "Pollster":"B", "Date":"Feb 10 2010"},
              {"ID":6, "State":"WA", "Pollster":"B", "Date":"Mar 22 2010"}]

In [11]:
most_recent_poll_row(poll_rows1, "A", "WA")

{'ID': 3, 'State': 'WA', 'Pollster': 'A', 'Date': 'Jan 08 2010'}

In [12]:
def unique_column_values(rows, column_name):
    """
    Given a list of rows and the name of a column (a string), returns a set
    containing all values in that column.
    """
    my_set = set()
    for i in rows:
        if i[column_name] not in my_set:
            my_set.add(i[column_name])
    return my_set
    

In [13]:
unique_column_values(poll_rows1, "State")

{'OR', 'WA'}

In [14]:
def pollster_predictions(poll_rows):
    """
    Given a list of poll data rows, returns pollster predictions.
    """

    dic = {}
    Pollster_set = unique_column_values(poll_rows, "Pollster")
    State_set = unique_column_values(poll_rows, "State")
    for i in Pollster_set:
        temp = []
        for j in State_set:
            a = most_recent_poll_row(poll_rows, i, j)
            if a is not None:
                temp.append(a)
                b = state_edges(temp)
                dic[i] = b            
    return dic


In [15]:
    rows3 = [
      {'State': 'WA', 'Dem': '1.0', 'Rep': '0.1', 'Date': 'Nov 05 2008', 'Pollster': 'PPP'},
      {'State': 'CA', 'Dem': '2.1', 'Rep': '3.2', 'Date': 'Nov 04 2008', 'Pollster': 'PPP'},
      {'State': 'WA', 'Dem': '9.1', 'Rep': '7.1', 'Date': 'Nov 05 2008', 'Pollster': 'IPSOS'},
      {'State': 'CA', 'Dem': '1.0', 'Rep': '10.3', 'Date': 'Nov 04 2008', 'Pollster': 'IPSOS'}]

In [16]:
pollster_predictions(rows3)

{'PPP': {'WA': 0.9, 'CA': -1.1}, 'IPSOS': {'WA': 2.0, 'CA': -9.3}}

In [17]:
def average_error(state_edges_predicted, state_edges_actual):
    """
    Given predicted state edges and actual state edges, returns
    the average error of the prediction.
    """
    error = 0
    count = 0
    predict_set = set(state_edges_predicted)
    actual_set = set(state_edges_actual)
    for name in predict_set.intersection(actual_set):
        error += abs(state_edges_predicted[name] - state_edges_actual[name])
        count += 1
    return (error/count)


In [18]:
    state_edges_pred_1 = {'WA': 1.0, 'CA': -2.3, 'ID': -20.1}
    state_edges_act_1 = {'WA': 2.1, 'CA': -1.4, 'ID': -19.1}

In [19]:
average_error(state_edges_pred_1, state_edges_act_1)

1.0

In [20]:
    state_edges_pred_3 = {'WA': 1.0, 'CA': 2.0}
    state_edges_act_3 = {'WA': 2.0, 'CA': 1.0, 'MA': 2.4, 'OR': -3.9}

In [21]:
average_error(state_edges_pred_3, state_edges_act_3)

1.0

In [22]:
def pollster_errors(pollster_predictions, state_edges_actual):
    """
    Given pollster predictions and actual state edges, retuns pollster errors.
    """
    dic = {}
    for innerDict in pollster_predictions:                        ###innerDict -> Pollster
        Pollster_predict = pollster_predictions[innerDict]        ###{'WA': 1.0, 'CA': -2.0, 'ID': -20.0}
        avg_err = average_error(Pollster_predict, state_edges_actual)
        dic.update({innerDict: avg_err})
    return dic

In [23]:
    predictions = {
      'PPP': {'WA': 1.0, 'CA': -2.0, 'ID': -20.0}, 
      'ISPOP': {'WA': 2.0, 'ID': -19.0} 
      }
    actual = {'WA': 2.0, 'CA': -1.0, 'ID': -19.0, 'OR': 2.2, 'DC': 0.1}

In [24]:
pollster_errors(predictions, actual)

{'PPP': 1.0, 'ISPOP': 0.0}

In [25]:
def pivot_nested_dict(nested_dict):
    """
    Pivots a nested dictionary, producing a different nested dictionary
    containing the same values.
    The input is a dictionary d1 that maps from keys k1 to dictionaries d2,
    where d2 maps from keys k2 to values v.
    The output is a dictionary d3 that maps from keys k2 to dictionaries d4,
    where d4 maps from keys k1 to values v.
    For example:
      input = { "a" : { "x": 1, "y": 2 },
                "b" : { "x": 3, "z": 4 } }
      output = {'y': {'a': 2},
                'x': {'a': 1, 'b': 3},
                'z': {'b': 4} }
    """
    from collections import defaultdict
    out = defaultdict(dict)
    for outer_key, inner in nested_dict.items():
        for inner_key, value in inner.items():
            out[inner_key][outer_key] = value
    return out

In [26]:
    us_wars_by_name = {
        "Revolutionary" : { "start": 1775, "end": 1783 },
        "Mexican" : { "start": 1846, "end": 1848 },
        "Civil" : { "start": 1861, "end": 1865 }
        }
    us_wars_by_start_and_end = {
        'start': {'Revolutionary': 1775, 'Civil': 1861, 'Mexican': 1846},
        'end': {'Revolutionary': 1783, 'Civil': 1865, 'Mexican': 1848}
        }
    print(pivot_nested_dict(us_wars_by_name) == us_wars_by_start_and_end)

True


In [27]:
def average_error_to_weight(error):
    """
    Given the average error of a pollster, returns that pollster's weight.
    The error must be a positive number.
    """
    return error ** (-2)

# The default average error of a pollster who did no polling in the
# previous election.
DEFAULT_AVERAGE_ERROR = 5.0

def pollster_to_weight(pollster, pollster_errors):
    """"
    Given a pollster and a pollster errors, return the given pollster's weight.
    """
    if pollster not in pollster_errors:
        weight = average_error_to_weight(DEFAULT_AVERAGE_ERROR)
    else:
        weight = average_error_to_weight(pollster_errors[pollster])
    return weight

In [28]:
def weighted_average(items, weights):
    """
    Returns the weighted average of a list of items.
    
    Arguments:
    items is a list of numbers.
    weights is a list of numbers, whose sum is nonzero.
    
    Each weight in weights corresponds to the item in items at the same index.
    items and weights must be the same length.
    """
    assert len(items) > 0
    assert len(items) == len(weights)
    total = 0
    sum_weights = 0
    for i in range(len(items)):
        total += items[i] * weights[i]
        sum_weights += weights[i]
    return(total/sum_weights)

In [29]:
    assert weighted_average([3, 4, 5], [1, 1, 1]) == 4
    assert weighted_average([3, 4], [1, 1]) == 3.5
    assert weighted_average([2, 4, 4, 6], [1, 1, 1, 5]) == 5
    assert weighted_average([0, 1, 2, 3, 4], [0, 1, 2, 3, 4]) == 3
    assert weighted_average([1, 2, 1], [3, 2, 5]) == 1.2


In [61]:
def average_edge(pollster_edges, pollster_errors):
    """
    Given pollster edges and pollster errors, returns the average of these edges
    weighted by their respective pollster errors.
    """
    from decimal import Decimal
    weight_sum = 0
    denum = 0
    weight = 0
    for i in pollster_edges:
        weight = Decimal(pollster_to_weight(i, pollster_errors))
        weight_sum += Decimal(weight)
        denum += Decimal(weight)*Decimal(pollster_edges[i]) 
    result = float(denum)/float(weight_sum)
    print(result)
    return result

In [62]:
    assert average_edge({"p1":3, "p2":4, "p3":5}, {"p1":1, "p2":1, "p3":1}) == 4
    assert average_edge({"p1":3, "p2":4, "p3":5}, {"p1":1, "p2":1, "p3":1, "p4":2, "p5": -8}) == 4
    assert average_edge({"p1":3, "p2":4}, {"p1":1, "p2":1}) == 3.5
    assert average_edge({"p1":2, "p2":4, "p3":4, "p4":6}, {"p1":1, "p2":1, "p3":1, "p4":5}) == 3.3684210526315788
    assert average_edge({"p1":1, "p2":2, "p3":3, "p4":4, "p5":5},
                        {"p1":1, "p2":2, "p3":3, "p4":4, "p5":5}) == 1.560068324160182
    assert average_edge({"p1":3, "p2":4, "p3":5}, {"p1":5, "p2":5}) == 4
    assert average_edge({"p1":3, "p2":4, "p3":5}, {}) == 4

4.0
4.0
3.5
3.3684210526315788
1.560068324160182
4.0
4.0


In [63]:
    pollster_predictions = {
      'PPP': { 'WA': -11.2, 'CA': -2.0, 'ID': -1.1 },
      'IPSOS': { 'WA': -8.7, 'CA': -3.1, 'ID': 4.0 },
      'SurveyUSA': { 'WA': -9.0, 'FL': 0.5 },
      }

In [64]:
dic_trans = pivot_nested_dict(pollster_predictions)
for i in dic_trans:
        edges = dic_trans[i]
        print(edges)
        #result = average_edge(edges, pollster_errors)
        #print(result)

{'PPP': -11.2, 'IPSOS': -8.7, 'SurveyUSA': -9.0}
{'PPP': -2.0, 'IPSOS': -3.1}
{'PPP': -1.1, 'IPSOS': 4.0}
{'SurveyUSA': 0.5}


In [68]:
def predict_state_edges(pollster_predictions, pollster_errors):
    """
    Given pollster predictions from a current election and pollster errors from
    a past election, returns the predicted state edges of the current election.
    """
    dic = {}
    dic_trans = pivot_nested_dict(pollster_predictions)
    for i in dic_trans:
        edges = dic_trans[i]
        result = average_edge(edges, pollster_errors)
        dic.update({ i: result})
    return dic
        

In [69]:
pollster_errors = {'PPP': 1.2, 'IPSOS': 4.0, 'SurveyUSA':3.5, 'NonExistant':100.0}
pollster_predictions = {
      'PPP': { 'WA': -11.2, 'CA': -2.0, 'ID': -1.1 },
      'IPSOS': { 'WA': -8.7, 'CA': -3.1, 'ID': 4.0 },
      'SurveyUSA': { 'WA': -9.0, 'FL': 0.5 },
      }

In [70]:
predict_state_edges(pollster_predictions, pollster_errors)

-10.799509886766941
-2.0908256880733944
-0.6788990825688075
0.5
{'WA': -10.799509886766941, 'CA': -2.0908256880733944, 'ID': -0.6788990825688075, 'FL': 0.5}
